In [106]:
# 문제1 ) simple linear regression
# 온도에 따른 ozone량 예측하는 프로그램을 TensorFlow를 이용하여 작성한다.
# 제공된 CSV 파일로 부터 온도와 온도에 따른 ozone량 데이터를 loading후 학습과정을 진행하고 
# 특정 온도(화씨 62도)에 따른 ozone량을 예측한다.

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action="ignore")

file_path = "./data/ozone/ozone.csv"
pd_ozone = pd.read_csv(file_path, sep=",")
# Ozone Temp 속성만을 갖는 df
pd_ozone_temp_orig = pd_ozone[["Ozone","Temp"]]
print(pd_ozone_temp.shape)
# 두 속성 중 결측 값 하나라도 있으면 행을 지움
pd_ozone_temp_orig.dropna(how="any", inplace=True)
print(pd_ozone_temp_orig.shape)


# 단위가 다르기때문에 일반화 시켜줌
# 0~1 사이의 값으로 나타냄
pd_ozone_temp = pd_ozone_temp_orig.apply(lambda x: (x - np.min(x))/(np.max(x) - np.min(x)), axis = 0)

# tensor graph
X = tf.placeholder(dtype = tf.float32)
Y = tf.placeholder(dtype = tf.float32)

W = tf.Variable(tf.random_normal([1,], dtype=tf.float32), name = "weight1")
b = tf.Variable(tf.random_normal([1,], dtype=tf.float32), name = "bias1")

H = W*X+b
cost = tf.reduce_mean(tf.square(H-Y))

optimizer = tf.train.AdamOptimizer(learning_rate=0.005)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


for step in range(5000):
    _, cost_value, weight_value, bias_value= sess.run([train,cost, W, b], feed_dict={X:pd_ozone_temp["Temp"], Y:pd_ozone_temp["Ozone"]})
    if step % 1000 == 0:
        print("cost_value : {}, weight_value : {}, bias_value : {}".format(cost_value,weight_value,bias_value))


#62도 일때 ozone  
predict_x = 62
x = pd_ozone_temp_orig["Temp"]
y = pd_ozone_temp_orig["Ozone"]
# temp를 일반화 시켜서 넣고
x_temp = (62 -  x.min(axis = 0))/(x.max(axis = 0) - x.min(axis = 0))
# y는 일반화의 역으로 뽑아봄
predict_y = (W * x_temp + b)*(y.max(axis=0)- y.min(axis=0))+y.min(axis=0)
print("{}도일때 ozone : {}".format(predict_x, sess.run(predict_y)))


(116, 2)
(116, 2)
cost_value : 1.656806468963623, weight_value : [-0.709326], bias_value : [-0.61831915]
cost_value : 0.022777212783694267, weight_value : [0.3540583], bias_value : [0.06949429]
cost_value : 0.01988827995955944, weight_value : [0.546398], bias_value : [-0.03757682]
cost_value : 0.01981690712273121, weight_value : [0.5805369], bias_value : [-0.05658038]
cost_value : 0.019816827028989792, weight_value : [0.5817219], bias_value : [-0.05724007]
62도일때 ozone : [3.5842187]
